# Assessing word embeddings for improving OCR accuracy

Our idea is to compare how different embeddings affect OCR accuracy, similar to the approach taken in this article: https://medium.com/states-title/using-nlp-bert-to-improve-ocr-accuracy-385c98ae174c (Links to an external site.)

We would like to try some embeddings we have learned about in the course, such as CBoW, word2vec, FastText, ELMo, BERT. If we have more time and find other interesting representations online we will evaluate them as well. The idea is to find a baseline OCR accuracy and then exploring if, and how much, this accuracy can be improved by applying word embeddings to the incorrectly scanned words. 

- Use a synthetic dataset (read in europarl corpus and corrupt it a bit)
- Richard suggests corrupting it in ways that often happen in OCR, like rn <-> m, i <-> l, cl <-> d 
    - (https://scribenet.com/articles/2016/03/04/how-to-get-the-most-out-of-ocr)
- I think he means we should use a spellchecker to find misspelled words, and check with NER that they are not a name
- Then maybe we don't throw out the misspelled word, it can be useful
- Richard thinks that character level embeddings can be useful here
- Evaluation of performance: see what people usually use within OCR. Some ideas:

    - https://www.aclweb.org/anthology/I17-1101.pdf
    - https://loicbarrault.github.io/papers/afli_cicling2015.pdf

In [124]:
import re
import spacy
import numpy as np
from enchant.checker import SpellChecker

spell = SpellChecker("en-UK")
nlp = spacy.load('en_core_web_sm')

from transformers import AutoTokenizer
from transformers import BertForMaskedLM
import torch

#import nltk
#from nltk.corpus import stopwords
#stop = stopwords.words('english')

## Try out spellchecker

In [2]:
spell.check("cat's")

True

In [5]:
spell.suggest("spel")

['spiel',
 'spelt',
 'spell',
 'seel',
 'spec',
 'sped',
 'spew',
 'Opel',
 'sp el',
 'sp-el']

In [6]:
# example - is this a good way to get most relevant word from enchant.spellcheck()?

import difflib
word="prfomnc"

dict,max = {},0
a = set(spell.suggest(word))

for b in a:
    
    tmp = difflib.SequenceMatcher(None, word, b).ratio();
    dict[tmp] = b
    
    if tmp > max:
        max = tmp

print(dict[max])

princedom


# Read in and corrupt the corpus

Character elision is when letter pairs and individual letters are confused by the software. These types of errors occur any time pairs of letters are shaped similarly to other letters. Six common pairs are:

rn <-> m, cl <-> d, vv <-> w, ol <-> d, li <-> h, nn <-> m

In [3]:
elisionArray = []
elisionArray.append(['rn', 'm'])
elisionArray.append(['ol', 'd'])
elisionArray.append(['cl', 'd'])
elisionArray.append(['vv', 'w'])
elisionArray.append(['li', 'h'])
elisionArray.append(['nn', 'm'])
elisionArray = np.array(elisionArray)

## An example of how the character elision will be handled

In [4]:
# A test sentence that contains a lot of 'rn' and 'li'
line = "I scorn this little barn with my lilac yarn and delicious fern"
print(line)

new_line = line
elision_prob = 0.5

# Randomize search order, since some letter pairs overlap 
elisionArray = np.random.permutation(elisionArray)
for pair in elisionArray:
    
    n_errors = 0
    
    for m in re.finditer(pair[0], new_line):
        
        rd = np.random.rand(1)
        if rd < elision_prob:
            
            # Note that the position can't be used later since the line changes length!
            print("--> Replaced ", pair[0], " at position ", m.start(), "")
            
            tmp = list(new_line)
            tmp[m.start()-n_errors:m.end()-n_errors] = "%%"
            new_line = "".join(tmp)
            new_line = new_line.replace("%%", pair[1])
            
            print(new_line)

            # count number of replacements
            n_errors += 1
            
print('-'*80)

# Save location and correct spelling for the corrupted words

line = list(line.split())
new_line = list(new_line.split())

total_errors = 0
ground_truth = []

# This will be the index of a document in the corpus
doc_num = 0     

# This contains the ground truth for each document 
tmp = []

for j in range(len(line)):
    if line[j] != new_line[j]:
        
        total_errors += 1
        tmp.append((j, line[j]))
        
ground_truth.append((doc_num, tmp))
print("Total errors: ", total_errors)
print(ground_truth)

I scorn this little barn with my lilac yarn and delicious fern
--> Replaced  rn  at position  22 
I scorn this little bam with my lilac yarn and delicious fern
--> Replaced  li  at position  13 
I scorn this httle bam with my lilac yarn and delicious fern
--> Replaced  li  at position  32 
I scorn this httle bam with my hlac yarn and delicious fern
--------------------------------------------------------------------------------
Total errors:  3
[(0, [(3, 'little'), (4, 'barn'), (7, 'lilac')])]


## Apply character elision to the Europarl data

### Function definitions

In [5]:
def check_line_errors(line, new_line, total_errors):
    
    """
    Check for errors between gold standard document and "OCR" document.
    """

    line = list(line.split())
    new_line = list(new_line.split()) 
    truth = []

    for j in range(len(line)):
        if line[j] != new_line[j]:

            total_errors += 1
            truth.append((j, line[j]))
    
    return truth, total_errors


def apply_elision(line, elision_array, elision_prob):
    
    """
    Apply elision to a line of text. An observed pair will be exchanged with probability elision_prob.
    """
    
    # Randomize search order, since some letter pairs overlap 
    elision_array = np.random.permutation(elision_array)
    for elision_pair in elision_array:

        # Count number of times each letter pair has been corrupted (since this changes the line length)
        n_errors = 0 

        for m in re.finditer(elision_pair[0], line):

            rd = np.random.rand(1)
            if rd < elision_prob:

                # Replace the letter pair and convert to a new line
                tmp = list(line)
                tmp[m.start()-n_errors:m.end()-n_errors] = "%%"
                line = "".join(tmp)
                line = line.replace("%%", elision_pair[1])

                # count number of replacements
                n_errors += 1

    return line


def read_data(corpus_file, corpus_encoding, max_lines, elision_array, elision_prob):
    
    """
    Read in ground truth version of the text, and a (synthetic) corrupted OCR dataset.
    """
    
    total_errors = 0
    ground_truth = []
    corrupted_data = []
    line_indices = []
    elision_errors = []
    
    with open(corpus_file, encoding = corpus_encoding) as f:
        
        for d, line in enumerate(f):
        
            if d == max_lines:
                break
        
            # Apply elision and keep track of which words have been corrupted
            new_line = apply_elision(line, elision_array, elision_prob)  
            line_truth, total_errors = check_line_errors(line, new_line, total_errors)
            
            if len(line_truth) > 0:
                line_indices.append(d)
                elision_errors.append(line_truth)
                
            # Append original and (potentially) corrupted line
            ground_truth.append(line) 
            corrupted_data.append(new_line)
                
    elisions = (line_indices, elision_errors)
            
    return ground_truth, corrupted_data, elisions, total_errors

In [6]:
def contains_numbers(string):
    
    """
    Check if the given string contains any digits.
    """
    
    return any(char.isdigit() for char in string)


def is_part_of_name(line, word):
    
    """
    Using spacy for NER, we check if a specific word is part of a name.
    """
    
    # Apply nlp pipeline, check if this "misspelled word" is a name
    result = nlp(line, disable = ['tagger', 'parser'])
    is_name = False

    for entity in result.ents:
        # If the "misspelled" word is part of the name of a person, country etc - we ignore it
        if entity.label_ in  ["PERSON", "NORP", "GPE", "ORG"] and entity.text.find(word) > -1:
            is_name = True

    return is_name


def identify_spelling_errors(data, ignore):

    """
    Given a dataset and a list of characters to ignore, find misspelled words that are _not_ names.
    """
    
    n_misspelled = 0
    line_indices = []
    spelling_errors = []
    spelling_corrected = []
    not_in_english = []

    for d, line in enumerate(data):

        words = line.split()
        tmp_1 = []
        tmp_2 = []
        
        for i, word in enumerate(words):

            # Some dates and similar are marked as misspelled - ignore words containing numbers!
            if not word in ignore and not contains_numbers(word):
            
                # Apply a spell checker
                if not spell.check(word):
        
                    # Check if the word is part of a name
                    if not is_part_of_name(line, word):

                        try:
                            # Apply a spell correction to the word
                            corrected_word = spell.suggest(word)[0]
                            
                            # Note down word and position
                            tmp_1.append((i, word))
                            tmp_2.append((i, corrected_word))
                            n_misspelled += 1

                        except IndexError:

                            # If no spelling suggestions exist, the line is usually not in English. 
                            # The EU parlaiment uses many languages, exclude the line just in case.
                            not_in_english.append(d)
        
        # If spelling errors were found, save the line and words.
        if len(tmp_1) > 0:
            line_indices.append(d)
            spelling_errors.append(tmp_1)
            spelling_corrected.append(tmp_2)
            
    spelling_errors = (line_indices, spelling_errors)
    spelling_corrected = (line_indices, spelling_corrected)

    return spelling_errors, spelling_corrected, n_misspelled, not_in_english

In [7]:
def remove_excl_lines(data, excl_lines):
    
    """
    Remove given lines, by index, from a dataset.
    """
    
    for i, line in enumerate(excl_lines):
        del data[line-i]

        
def remove_excl_entries(entries, count, excl_lines):
    
    """
    Given lines to exclude, by index, remove them from the data structure at the right position. Also adjust count.
    """
    
    a = set(excl_lines)
    b = set(entries[0])
    
    for line_idx in a & b:
            
        # Find position of the line to be removed
        list_pos = entries[0].index(line_idx)
        
        # Adjust the word count
        count -= len(entries[1][list_pos])

        # Delete the identified entries
        del entries[0][list_pos]
        del entries[1][list_pos]
          
    return entries, count

### Run the code

In [8]:
file = "europarl.txt"
encoding = "utf-8"

# Probability that a letter pair from our list will be confused if it is seen in the text
elision_prob = 0.1
max_lines = 10000

np.random.seed(0)
ground_truth, data, elisions, n_elisions = read_data(file, encoding, max_lines, elisionArray, elision_prob)

In [9]:
# Words and characters for the spellchecker to ignore
ignore = [",", ".", '"', "(", ")", "-", "'", "!", "?", ":", ";", "/", "n't", "'s", "'m", "%", "--", "``", "___LANGCODE___"]
spell_errors, spell_corrected, n_misspelled, excl_lines = identify_spelling_errors(data, ignore)

In [10]:
print("Number of identified misspelled words: ", n_misspelled)
print("Number of synthetic errors: ", n_elisions)

Number of identified misspelled words:  1995
Number of synthetic errors:  1190


While spell checking, some words might appear that have no spelling suggestions. These words tend to be in French or German, since the EU parliament uses these languages occasionally in quotes. It's not fair to ask the spell checker to handle these words, and the purpose here is not to handle multiple different languages. Thus we will remove any lines that are not only in English (in the first 10 000 documents of the corpus, only 6 are affected). 

If there are entries in excl_lines:
- Remove the excl_lines from ground_truth and data
- If any of the excl_lines appear in elisions, elisions and n_elisions need to be adjusted
- If any of the excl_lines appear in spell_errors or spell_corrected, adjust them and n_misspelled

In [11]:
# If there are entries in excl_lines, make adjustments as described above.

remove_excl_lines(data, excl_lines)
remove_excl_lines(ground_truth, excl_lines)

elisions, n_elisions = remove_excl_entries(elisions, n_elisions, excl_lines)
spell_errors, n_misspelled = remove_excl_entries(spell_errors, n_misspelled, excl_lines)
spell_corrected, _ = remove_excl_entries(spell_corrected, n_misspelled, excl_lines)

# Compute baseline accuracy using spellcheck suggestions

baseline: use top suggested word from enchant spellchecker

so I guess we create a list of misspelled words just like doc_errors, apply the suggestion for each, and see if we get closer to the ground truth or not?

- https://www.aclweb.org/anthology/I17-1101.pdf

[From the article] For evaluation, we use the CER (Character Error Rate) metric as defined in OCR post-correction evaluations:

$$ CER = \frac{S + D + I}{S + D + C} $$

Where S refers to the number of substituted characters in the OCR text (w.r.t. the reference texts), D to the number of deleted characters, I to the number of inserted characters and C to the number of ‘correct’ characters. We use the CER metric when comparing to the baseline system. __Is this useful in the character elision setting? Doesn't look like it__

We want to measure the systems performance per input, rather than per character. We therefore introduce two complementary accuracy-based evaluation metrics, which evaluate on the level of the character window:
- detection accuracy (detAcc) shows the proportion of correctly detected errors and nonerrors in the evaluated set of 20-character strings
- correction accuracy (corrAcc) reflects the ability of the language model to accurately correct corrupted strings without overgenerating and editing non-corrupted strings

These metrics are calculated as follows:

$detAcc = \frac{(TP + TN + incorrectEdit)}{(TP + TN + FP + FN + incorrectEdit)}$

$corrAcc = \frac{(TP + TN)}{(TP + TN + FP + FN + incorrectEdit)}$

- TP: There is an error on the line, which is corrected.
- TN: No error on the line, no correction
- FP: No error on the line, makes a correction anyway
- FN: There is an error on the line, but it's not corrected
- incorrectEdit: Error is identified and corrected, but other words are also corrected which should not be (within the same document). Basically, anything that doesn't fall into the other categories.

- https://loicbarrault.github.io/papers/afli_cicling2015.pdf

They use BLEU. The Bilingual Evaluation Understudy Score, or BLEU for short, is a metric for evaluating a generated sentence to a reference sentence. A perfect match results in a score of 1.0, whereas a perfect mismatch results in a score of 0.0

Also, they use WER (word error rate). To get the WER, start by adding up the substitutions, insertions, and deletions that occur in a sequence of recognized words. Divide that number by the total number of words originally spoken. The result is the WER. To put it in a simple formula, Word Error Rate = (Substitutions + Insertions + Deletions) / Number of Words Spoken

But how do you add up those factors? Let’s look at each one:

- A substitution occurs when a word gets replaced (for example, “noose” is transcribed as “moose”)
- An insertion is when a word is added that wasn’t said (for example, “SAT” becomes “essay tea”)
- A deletion happens when a word is left out of the transcript completely (for example, “turn it around” becomes “turn around”)

Since we don't consider insertions and deletions, WER doesn't seem very relevant either. 

In [177]:
# From one of the articles
true_line = "In oven for 15 minutes"

# True positive case
line_in  = "In o5en for 15 minutes"
line_out = "In oven for 15 minutes"

# True negative case
line_in  = "In oven for 15 minutes"
line_out = "In oven for 15 minutes"

# False positive case
line_in  = "In oven for 15 minutes"
line_out = "In oven for 15 m1nutes"

# False negative case
line_in  = "In o5en for 15 minutes"
line_out = "In o5en for 15 minutes"

# Incorrect edit case
line_in  = "In o5en for 15 minutes"
line_out = "In oven for 15 m1nutes"

In [12]:
def compute_accuracy_scores(elisions, corrections, data):
    
    elision_idx = elisions[0]
    elision_words = elisions[1]
    corrected_idx = corrections[0]
    corrected_words = corrections[1]
    
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    incorrect_edits = 0
    
    # True positives = number of correct changes
    for pos1, idx in enumerate(elision_idx):
        if idx in corrected_idx:
            
            pos2 = corrected_idx.index(idx)
            
            if elision_words[pos1] == corrected_words[pos2]:
                TP += 1
            else:
                incorrect_edits += 1

    print("True positives: ", TP)  
    
    # True negatives = number of lines that do not have elision, and that have not been touched by spellchecker
    corpus_size = len(data)
    all_touched_lines = set(corrected_idx) | set(elision_idx)
    TN = corpus_size - len(all_touched_lines)
    print("True negatives: ", TN)
    
    # False negatives = lines that should have been changed, but were not
    false_neg_lines = set(elision_idx) - set(corrected_idx)
    FN = len(false_neg_lines)
    print("False negatives: ", FN)
    
    # False positives = there was no elision, but a correction was made anyway
    false_pos_lines = set(corrected_idx) - set(elision_idx)
    FP = len(false_pos_lines)
    print("False positives: ", FP)
    
    # Instances when the right line has been edited, but incorrectly
    print("Incorrect edits: ", incorrect_edits)
    
    # Double check that it all adds up
    if FP + FN + TN + TP + incorrect_edits != corpus_size:
        print("ERROR: These scores don't add up to the number of lines!")
        
    # Compute and display accuracy scores    
    det_accuracy = (TP + TN + incorrect_edits)/corpus_size
    corr_accuracy = (TP + TN)/corpus_size
    
    print("-"*75)
    print("Detection accuracy: ", np.round(det_accuracy, 4))
    print("Correction accuracy: ", np.round(corr_accuracy, 4))
    
    return det_accuracy, corr_accuracy
        

In [13]:
det_acc, corr_acc = compute_accuracy_scores(elisions, spell_corrected, data)

True positives:  28
True negatives:  8192
False negatives:  108
False positives:  692
Incorrect edits:  974
---------------------------------------------------------------------------
Detection accuracy:  0.92
Correction accuracy:  0.8225


# Try to fill in missing words using BERT 
(like in the article)

In [205]:
# Find a good example document to use BERT on

example_idx = 260

print(spell_errors[0][example_idx], spell_errors[1][example_idx])

line_idx = spell_errors[0][example_idx]
line_info = spell_errors[1][example_idx]

line = data[line_idx]
print(line)

1599 [(1, 'concems')]
However , when health becomes an object of trade and exploitation , when large private clinics reap huge profits from groups of patients who need a transplant , there can be no terms or conditions to the safeguarding of transparency in organ donation .



In [201]:
# Replace misspelled words with [MASK]
split_line = line.split()

for item in line_info:
    word = split_line[item[0]]
    line = line.replace(word, '[MASK]')

print(line)

On the other hand , I believe that perhaps one of the amendments presented by the Committee on Foreign Affairs is rather over the top insofar as it could constitute a first step towards the creation of a European Union foreign service , but it is the case that these [MASK] officers , with the support of the offices of the Commission in relation to information and coordination , are without doubt a first step towards the creation of this Community immigration [MASK] which we all want to see .



In [202]:
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [203]:
# Load, train and predict using pre-trained model
tokenized_text = tokenizer.tokenize(line)
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
MASKIDS = [i for i, e in enumerate(tokenized_text) if e == '[MASK]']

# Create the segments tensors
segs = [i for i, e in enumerate(tokenized_text) if e == "."]
segments_ids=[]
prev=-1
for k, s in enumerate(segs):
    segments_ids = segments_ids + [k] * (s-prev)
    prev=s
segments_ids = segments_ids + [len(segs)] * (len(tokenized_text) - len(segments_ids))
segments_tensors = torch.tensor([segments_ids])

# prepare Torch inputs 
tokens_tensor = torch.tensor([indexed_tokens])
# Load pre-trained model
model = BertForMaskedLM.from_pretrained('bert-base-uncased').eval()
# Predict all tokens
with torch.no_grad():
    predictions = model(tokens_tensor, segments_tensors)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [204]:
mask_predictions = []
n_guesses = 5

for mask_idx in MASKIDS:
    
    word_int = predictions[0][0, mask_idx, :].topk(5).indices.tolist()
    print(tokenizer.convert_ids_to_tokens(word_int))


['two', 'three', 'new', 'these', 'four']
[',', 'service', 'system', '-', 'office']


In [199]:
# Example from https://github.com/renatoviolin/next_word_prediction/blob/master/main.py

def decode(tokenizer, pred_idx, top_clean):
    ignore_tokens = string.punctuation + '[PAD]'
    tokens = []
    for w in pred_idx:
        token = ''.join(tokenizer.decode(w).split())
        if token not in ignore_tokens:
            tokens.append(token.replace('##', ''))
    return '\n'.join(tokens[:top_clean])


def encode(tokenizer, text_sentence, add_special_tokens=True):
    text_sentence = text_sentence.replace('[MASK]', tokenizer.mask_token)
    # if <mask> is the last token, append a "." so that models dont predict punctuation.
    if tokenizer.mask_token == text_sentence.split()[-1]:
        text_sentence += ' .'

    input_ids = torch.tensor([tokenizer.encode(text_sentence, add_special_tokens=add_special_tokens)])
    mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
    return input_ids, mask_idx

# ========================= BERT =================================

import torch
import string

from transformers import BertTokenizer, BertForMaskedLM
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertForMaskedLM.from_pretrained('bert-base-uncased').eval()

text_sentence = line
top_k = 10
top_clean=5

print(text_sentence)
input_ids, mask_idx = encode(bert_tokenizer, text_sentence)
with torch.no_grad():
    predict = bert_model(input_ids)[0]
bert = decode(bert_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In its report , the Court of Auditors puts forward the view that there was a cumulation of subsidies and irregularities in the [MASK] of the programmes because the measures overlapped .



# Try using bigrams 
This could catch when an incorrectly scanned word becomes another, correctly spelled word: for example, God -> Cod. The spell checker will not react to this, but it should create a strange bigram

# Try some other word embeddings

two approaches: either throw away the misspelled word and try to fill it in from context, or use character level embeddings 

# Try some character level embeddings